In [2]:
import pandas as pd
import requests as rq
import numpy as np
from bs4 import BeautifulSoup as bsp
import time
import re

In [3]:
## movie details page test
for i in [45997,229733]:
    try:
        detailpage=rq.get("http://movie.mtime.com/"+i,timeout=0.1)
    except:
        print(f'{i} Time out')

45997 Time out
229733 Time out


In [ ]:
detailpage=rq.get("http://movie.mtime.com/229733",timeout=10)
print(detailpage.status_code)
soup=bsp(detailpage.text,"lxml")
## Runtime, Genre(catergory), ReleaseDate, FilmFormat
more_infor=soup.body.find_all("div",{"class":"otherbox"})
more_infor[0].text.split("-")

In [5]:
detailpage=rq.get("http://movie.mtime.com/229733/fullcredits.html",timeout=10)
print(detailpage.status_code)
soup=bsp(detailpage.text,"lxml")
## 演、职员
actor_sec=soup.body.find_all("div",{"class":"actor_tit"})
people={}
actor_name=[]
for i in actor_sec:
    if i.h4:
        actor=i.h4.text.strip()
    else:
        actor_name.append(i.text.strip())
people[actor]=actor_name 
## Staff member
staff_sec=soup.body.find_all("div",{"class":"credits_r"})[0].find_all("div",{"class":"credits_list"})
for i in staff_sec:
    title=(i.h4.text.strip())
    member_list=[]
    member=i.find_all('p')
    if member:
        for p in member:
            member_list.append(p.text.strip())
    people[title]=member_list
people

200


{'演员 Actor': ['吴京 Jing Wu',
  '弗兰克·格里罗 Frank Grillo',
  '吴刚 Gang Wu',
  '张翰 Han Zhang',
  '卢靖姗 Celina Jade',
  '丁海峰 Haifeng Ding',
  '余男 Nan Yu',
  '于谦 Qian Yu',
  'Nwachukwu Kennedy ..',
  'Diana Sylla',
  '石兆琪 Zhaoqi Shi',
  '淳于珊珊 Shanshan Chunyu',
  '倪大红 Dahong Ni',
  '肖竹 Zhu Xiao',
  'Qiang Ma',
  'Zibin Fang',
  'Sen Wang',
  'Tengyuan Liu',
  'Yongda Zhang',
  'Xiaolong Zhang',
  'Zi Liang',
  'Guangping Guo',
  '茹萍 Ping Ru',
  'Doudou',
  'Zhang Heng',
  'Miaomiao Tian',
  'Yuanyuan Shan',
  'Mao San',
  '斯科特·阿特金斯 Scott Adkins',
  '凯文·李 Kevin Lee',
  'Christopher Collins',
  '索纳·伊姆贝 Sona Eyambe',
  '凯尔·夏皮罗 Kyle Shapiro',
  '塞缪尔·西维尔奇 Samuel Thivierge',
  'Alberto Bossum',
  'Afshin',
  '邓紫衣 Ziyi Deng',
  '赵毅 Yi Zhao'],
 '导演 Director': ['吴京', 'Jing Wu'],
 '编剧 Writer': ['吴京 Jing Wu', '刘毅 Yi Liu', '董群 Qun Dong'],
 '制作人 Produced by': ['关海龙 Hailong Guan - 制片人',
  '张苗 Miao Zhang - 制片人',
  '嵇道青 Daoqing Ji - 制片人',
  '吴京 Jing Wu - 出品人',
  '吕建民 Jianmin Lv - 出品人',
  '徐子泉 Ziquan Xu - 出品人',
 

In [6]:
detailpage=rq.get("http://movie.mtime.com/196613/awards.html",timeout=10)
detailpage.status_code
# 229733 
# 262215
# 242167
# 196613

200

In [7]:
## 奖项
soup=bsp(detailpage.text,"lxml")
award_name=soup.body.find_all("h3")
award_detail=soup.body.find_all("dl")
aw_names=[]
if award_name:
    for i in award_name:
        if i.b:
            aw_names.append(i.text.strip())
        else:
            award_summary=i.text.strip()
for i in range(len(award_detail)):
    if 'id' in award_detail[i].attrs:
        if award_detail[i]['id']=="awardSlidesItems":
            position=i+1
            break
details=award_detail[position:]
assert len(details)==len(aw_names)
print(award_summary)
print(aw_names)
details[0]

本片共获金球奖等重要奖项3次，提名10次
['美国金球奖 (2016；第73届) 提名：1', 'MTV电影奖 (2016；第25届) 提名：2', '土星奖 (2016；第42届) 获奖：1; 提名：2', '青少年选择奖 (2015；第17届) 获奖：2; 提名：5']


<dl> <dt>提名</dt> <dd> <i class="dot">·</i><span>电影类-最佳原创歌曲</span> <a href="http://people.mtime.com/1937853/" target="_blank">维兹·卡利法 Wiz Khalifa</a> <a href="http://people.mtime.com/1919917/" target="_blank"> Justin Franks</a> <a href="http://people.mtime.com/2180024/" target="_blank"> Charlie Puth</a> <a href="http://people.mtime.com/2180025/" target="_blank"> Andrew Cedar</a> </dd> </dl>

In [8]:
## for each award, get detail
award_dict=dict()
for i in range(len(aw_names)):
    name=re.search(r'\w+奖',aw_names[i]).group()
    summary=re.search(r'奖 \(.+',aw_names[i]).group().replace(r"奖 ","")
    award_dict[name]=[summary,re.sub(r'·\w+-','#',details[i].text.strip())]
award_dict

{'美国金球奖': ['(2016；第73届) 提名：1',
  '提名  #最佳原创歌曲 维兹·卡利法 Wiz Khalifa  Justin Franks  Charlie Puth  Andrew Cedar'],
 'MTV电影奖': ['(2016；第25届) 提名：2', '提名  #最佳群戏   #最佳动作演员 范·迪塞尔 Vin Diesel'],
 '土星奖': ['(2016；第42届) 获奖：1; 提名：2',
  '获奖  #最佳动作/冒险电影  提名  #最佳剪辑  Leigh Folsom  Kirk M. Morri 克里斯蒂安·瓦格纳 Christian Wagner  Dylan Highsmith   #最佳DVD/蓝光特别版'],
 '青少年选择奖': ['(2015；第17届) 获奖：2; 提名：5',
  '获奖  #最佳动作电影男演员 保罗·沃克 Paul Walker   #最佳动作电影  提名  #最佳动作电影男演员 范·迪塞尔 Vin Diesel   #最佳动作电影女演员 米歇尔·罗德里格兹 Michelle Rodriguez   #最佳动作电影女演员 乔丹娜·布鲁斯特 Jordana Brewster   #最佳电影火花 范·迪塞尔 Vin Diesel 乔丹娜·布鲁斯特 Jordana Brewster 道恩·强森 Dwayne Johnson 卢达·克里斯 Ludacris 米歇尔·罗德里格兹 Michelle Rodriguez 泰瑞斯·吉布森 Tyrese Gibson 保罗·沃克 Paul Walker   #最佳电影反派 杰森·斯坦森 Jason Statham']}

In [9]:
## check companies
detailpage=rq.get("http://movie.mtime.com/229733/details.html",timeout=10)
detailpage.status_code
# 229733 
# 262215
# 242167
# 196613


200

In [19]:
soup=bsp(detailpage.text,"lxml")
company_sec=soup.body.find_all("dl",{"id":"companyRegion"})
if company_sec:
    company_detail=company_sec[0].find_all("div",{"class":"fl wp49"})
    assert len(company_detail)==2

In [18]:
company_sec[0].find_all("div",{"class":"fl wp49"})

[<div class="fl wp49"> <h4>制作公司</h4> <ul> <li>1.  <a href="http://movie.mtime.com/company/120305/" target="_blank">北京登峰国际文化传播有限公司</a> <span>  </span></li> <li>2.  <a href="http://movie.mtime.com/company/139097/" target="_blank">嘲风影业（北京）有限公司</a> <span>[中国]  </span></li> <li>3.  <a href="http://movie.mtime.com/company/133995/" target="_blank">霍尔果斯春秋时代文化传媒有限公司</a> <span>[中国]  </span></li> <li>4.  <a href="http://movie.mtime.com/company/136770/" target="_blank">捷成世纪文化产业集团有限公司</a> <span>  </span></li> <li>5.  <a href="http://movie.mtime.com/company/134200/" target="_blank">霍尔果斯橙子映像传媒有限公司</a> <span>[中国]  </span></li> <li data-selector="hideMoreCompay" style="display:none;">6.  <a href="http://movie.mtime.com/company/133994/" target="_blank">霍尔果斯登峰国际文化传播有限公司</a> <span>[中国]  </span> </li> <li data-selector="hideMoreCompay" style="display:none;">7.  <a href="http://movie.mtime.com/company/102465/" target="_blank">中国电影股份有限公司</a> <span>[中国]联合出品  </span> </li> <li data-selector="hideMoreCompay" st